In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import os
os.chdir('../')
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
from src.dataset import *
from src.concept_vectors import *
from src.util import *
from src.hierarchy import *
from src.metrics import *
import numpy as np
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
from src.create_vectors import *

/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

In [12]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]
        )
    except RuntimeError as e:
        print(e)
else:
    print("No GPus")


No GPus


In [5]:
seeds = [43,44,45]
vector_method = [load_label_vectors_simple,load_shapley_vectors_simple,load_concept2vec_vectors_simple,load_cem_vectors_simple]
vector_names = ["Label","Shapley","Concept2Vec","CEM"]

## Debugging: Check which TCAV Vectors are available

In [7]:
dataset = CUB_Dataset()
attributes = dataset.get_attributes()

In [24]:
def attribute_exists(attribute_name,suffix,seed):
    attribute_name = attribute_name.replace(" ","_")
    folder_name = "results/cavs/cub{}/{}/".format(suffix,seed)
    for i in range(3):
        file_name = "{}-random500_{}-block4_conv1-linear-0.1.pkl".format(attribute_name,i)
        
        if not os.path.exists(folder_name + file_name):
            print(folder_name+file_name)
            return False
    return True

In [27]:
for attribute in attributes:
    for seed in [43,44,45]:
        for suffix in ['','_image_robustness','_image_responsiveness']:
            attribute_exists(attribute,suffix,seed)

## Evaluate All Vectors MNIST

In [10]:
dataset = MNIST_Dataset()
attributes = dataset.get_attributes()

In [15]:
for method,name in zip(vector_method,vector_names):
    print("Computing {}".format(name))
    start = time.time()

    results = compute_all_metrics(method,
                                        dataset,
                                        attributes,
                                        seeds)
    print("Method {}: {}".format(name,results))
    
    name_lower = name.lower()
    w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
    for key in results:
        w.write("{}: {}\n".format(key,results[key]))
    w.close()


Computing Label
Stability: (0.0, 0.0)
Image Robustness: (0.04761904761904767, 0.0)
Image Responsiveness: (1.0, 0.0)


/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 10000 validated image filenames belonging to 10 classes.
313/313 [==============================] - 29s 90ms/step
Truthfulness: (0.9523809523809522, 1.1102230246251565e-16)
Method Label: {'Stability': (0.0, 0.0), 'Image Robustness': (0.04761904761904767, 0.0), 'Image Responsiveness': (1.0, 0.0), 'Truthfulness': (0.9523809523809522, 1.1102230246251565e-16)}
Computing TCAV
Stability: (0.03174603174603178, 0.022447834323382487)
Image Robustness: (0.9047619047619048, 0.0)
Image Responsiveness: (1.0, 0.0)


/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 10000 validated image filenames belonging to 10 classes.
313/313 [==============================] - 29s 91ms/step


...

Found 10000 validated image filenames belonging to 10 classes.
313/313 [==============================] - 29s 94ms/step
Truthfulness: (0.9523809523809522, 1.1102230246251565e-16)
Method Concept2Vec: {'Stability': (0.04761904761904767, 0.0), 'Image Robustness': (0.04761904761904767, 0.0), 'Image Responsiveness': (0.9682539682539683, 0.022447834323382487), 'Truthfulness': (0.9523809523809522, 1.1102230246251565e-16)}
Computing CEM
Stability: (0.253968253968254, 0.14720029357929687)
Image Robustness: (0.28571428571428575, 0.10286889997472791)
Image Responsiveness: (0.9523809523809524, 0.06734350297014735)


/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 10000 validated image filenames belonging to 10 classes.
313/313 [==============================] - 28s 89ms/step


/auto/homes/njr61/main_code/src/metrics.py:361: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


Truthfulness: (0.8253968253968255, 0.17958267458705968)
Method CEM: {'Stability': (0.253968253968254, 0.14720029357929687), 'Image Robustness': (0.28571428571428575, 0.10286889997472791), 'Image Responsiveness': (0.9523809523809524, 0.06734350297014735), 'Truthfulness': (0.8253968253968255, 0.17958267458705968)}


## Evaluate all Vectors CUB

In [29]:
dataset = CUB_Dataset()
attributes = dataset.get_attributes()
seeds = [43,44,45]

In [30]:
for method,name in zip([load_tcav_vectors_simple]+vector_method,['TCAV']+vector_names):
    print("Computing {}".format(name))
    start = time.time()

    results = compute_all_metrics(method,
                                        dataset,
                                        attributes,
                                        seeds)
    print("Method {}: {}".format(name,results))
    
    name_lower = name.lower()
    w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
    for key in results:
        w.write("{}: {}\n".format(key,results[key]))
    w.close()
    

Computing TCAV
Stability: (0.9890873015873017, 0.0037119616932281036)
Image Robustness: (0.9970238095238096, 0.002430049347999176)
Image Responsiveness: (0.9880952380952381, 0.004860098695998397)


2023-04-30 15:34:21.521337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 199s 5s/step


/auto/homes/njr61/main_code/src/metrics.py:358: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_concept_embeddings = np.array([embedding_method(i,dataset,"",seed=seed) for i in dataset.get_attributes()])


Truthfulness: (0.0755952380952381, 0.008908708063747483)
Method TCAV: {'Stability': (0.9890873015873017, 0.0037119616932281036), 'Image Robustness': (0.9970238095238096, 0.002430049347999176), 'Image Responsiveness': (0.9880952380952381, 0.004860098695998397), 'Truthfulness': (0.0755952380952381, 0.008908708063747483)}


## Evalaute all Vectors DSprites

In [6]:
dataset = DSprites_Dataset()
attributes = dataset.get_attributes()

In [7]:
for method,name in zip(vector_method,vector_names):
    print("Computing {}".format(name))
    start = time.time()

    results = compute_all_metrics(method,
                                        dataset,
                                        attributes,
                                        seeds)
    print("Method {}: {}".format(name,results))
    
    name_lower = name.lower()
    w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
    for key in results:
        w.write("{}: {}\n".format(key,results[key]))
    w.close()

Computing Label
Stability: (0.0, 0.0)
Image Robustness: (0.8148148148148149, 0.0)
Image Responsiveness: (0.7777777777777778, 0.0)


2023-04-28 22:24:23.156494: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 750 validated image filenames belonging to 9 classes.
24/24 [==============================] - 123s 5s/step


/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


Truthfulness: (0.5, 0.0)
Method Label: {'Stability': (0.0, 0.0), 'Image Robustness': (0.8148148148148149, 0.0), 'Image Responsiveness': (0.7777777777777778, 0.0), 'Truthfulness': (0.5, 0.0)}
Computing Shapley
Stability: (0.04938271604938275, 0.008729713347982055)
Image Robustness: (0.38888888888888884, 0.01512030705421717)
Image Responsiveness: (0.5679012345679012, 0.061107993435874475)


...

Truthfulness: (0.3074074074074075, 0.027715980642769932)
Method Concept2Vec: {'Stability': (0.1666666666666667, 0.026189140043946214), 'Image Robustness': (0.8765432098765432, 0.04364856673991033), 'Image Responsiveness': (0.8395061728395062, 0.031475429096251756), 'Truthfulness': (0.3074074074074075, 0.027715980642769932)}
Computing CEM
Stability: (0.29012345679012347, 0.031475429096251756)
Image Robustness: (0.7592592592592592, 0.15930231976004866)
Image Responsiveness: (0.8765432098765432, 0.05310077325334951)


/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 750 validated image filenames belonging to 9 classes.
24/24 [==============================] - 123s 5s/step


/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/auto/homes/njr61/main_code/src/metrics.py:361: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_concept_embeddings = np.array([embedding_method(i,dataset,"",seed=seed) for i in dataset.get_attributes()])


Truthfulness: (0.2703703703703704, 0.010475656017578472)
Method CEM: {'Stability': (0.29012345679012347, 0.031475429096251756), 'Image Robustness': (0.7592592592592592, 0.15930231976004866), 'Image Responsiveness': (0.8765432098765432, 0.05310077325334951), 'Truthfulness': (0.2703703703703704, 0.010475656017578472)}


## Evaluate all Vectors Chexpert

In [8]:
dataset = Chexpert_Dataset()
attributes = dataset.get_attributes()

In [9]:
for method,name in zip(vector_method,vector_names):
    print("Computing {}".format(name))
    start = time.time()

    results = compute_all_metrics(method,
                                        dataset,
                                        attributes,
                                        seeds)
    print("Method {}: {}".format(name,results))
    
    name_lower = name.lower()
    w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
    for key in results:
        w.write("{}: {}\n".format(key,results[key]))
    w.close()

Computing Label
Stability: (0.0, 0.0)
Image Robustness: (0.07692307692307687, 0.0)
Image Responsiveness: (0.794871794871795, 1.1102230246251565e-16)


/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 1000 validated image filenames belonging to 2 classes.
32/32 [==============================] - 165s 5s/step
Truthfulness: (0.5076923076923078, 0.0)
Method Label: {'Stability': (0.0, 0.0), 'Image Robustness': (0.07692307692307687, 0.0), 'Image Responsiveness': (0.794871794871795, 1.1102230246251565e-16), 'Truthfulness': (0.5076923076923078, 0.0)}
Computing Shapley
Stability: (0.03418803418803421, 0.024174590809796514)
Image Robustness: (0.6324786324786325, 0.03197997766473456)
Image Responsiveness: (0.7008547008547009, 0.0792616965426983)


/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 1000 validated image filenames belonging to 2 classes.
32/32 [==============================] - 165s 5s/step
Truthfulness: (0.46666666666666673, 0.026148818018424547)
Method Shapley: {'Stability': (0.03418803418803421, 0.024174590809796514), 'Image Robustness': (0.6324786324786325, 0.03197997766473456), 'Image Responsiveness': (0.7008547008547009, 0.0792616965426983), 'Truthfulness': (0.46666666666666673, 0.026148818018424547)}
Computing Concept2Vec
Stability: (0.5726495726495726, 0.052687299170675)
Image Robustness: (0.358974358974359, 0.07548513560963974)
Image Responsiveness: (0.7606837606837606, 0.03197997766473456)


...

Found 1000 validated image filenames belonging to 2 classes.
32/32 [==============================] - 165s 5s/step
Truthfulness: (0.4102564102564103, 0.06322475900480998)
Method Concept2Vec: {'Stability': (0.5726495726495726, 0.052687299170675), 'Image Robustness': (0.358974358974359, 0.07548513560963974), 'Image Responsiveness': (0.7606837606837606, 0.03197997766473456), 'Truthfulness': (0.4102564102564103, 0.06322475900480998)}
Computing CEM
Stability: (0.5555555555555557, 0.06729921259839156)
Image Robustness: (0.5042735042735043, 0.11530544925839355)
Image Responsiveness: (0.7264957264957265, 0.12791991065893818)


/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 1000 validated image filenames belonging to 2 classes.
32/32 [==============================] - 165s 5s/step


/auto/homes/njr61/main_code/src/metrics.py:361: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_concept_embeddings = np.array([embedding_method(i,dataset,"",seed=seed) for i in dataset.get_attributes()])


Truthfulness: (0.4153846153846154, 0.03768445758127969)
Method CEM: {'Stability': (0.5555555555555557, 0.06729921259839156), 'Image Robustness': (0.5042735042735043, 0.11530544925839355), 'Image Responsiveness': (0.7264957264957265, 0.12791991065893818), 'Truthfulness': (0.4153846153846154, 0.03768445758127969)}


## CUB Ablation Studies

### Evaluation at different noise levels

In [17]:
dataset = CUB_Dataset()
attributes = dataset.get_attributes()

In [22]:
embedding_method = load_shapley_vectors_simple
name = "Shapley"
random_seeds = [43]

In [23]:
start = time.time()

results = {}

for flip_prob in ['0.01','0.05','0.1']:
    results['flip_prob_{}'.format(flip_prob)] = compare_same_images_by_suffix(embedding_method,
                                         dataset,attributes,random_seeds,"_flip_{}".format(flip_prob),
                                        baseline_hierarchies=None)

for noise in [25,50,100]:
    results['noise_{}'.format(flip_prob)] = compare_same_images_by_suffix(embedding_method,
                                         dataset,attributes,random_seeds,"_noise_{}".format(noise),
                                        baseline_hierarchies=None)

w = open("results/evaluation/cub_noise_ablation.txt","w")
for key in results:
    w.write("{}: {}\n".format(key,results[key]))
w.close()

### Evalaution with different models

In [34]:
for model_name in ["Resnet50","InceptionV3"]:
    for s in [43,44,45]:
        create_shapley_vectors(attributes,dataset,"",seed=s,model_name=model_name)

Using directory results/shapley/cub_model_resnet50/43
Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 336s 2s/step
Using directory results/shapley/cub_model_resnet50/44
Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 344s 2s/step
Using directory results/shapley/cub_model_resnet50/45
Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 339s 2s/step
Using directory results/shapley/cub_model_inceptionv3/43
Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 177s 1s/step
Using directory results/shapley/cub_model_inceptionv3/44
Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 176s 1s/step
Using directory results/shapley/cub_model_inceptionv3/45
Found 4796 validated image filenames belonging to 200 classes.
150/150 [

In [36]:
random_seeds = [43,44,45]
results = {}

for model_name in ['resnet50','inceptionv3']:
    results[model_name] = compare_same_images_by_suffix(embedding_method,
                                         dataset,attributes,random_seeds,"_model_{}".format(model_name),
                                        baseline_hierarchies=None)

w = open("results/evaluation/cub_model_comparison.txt","w")
for key in results:
    w.write("{}: {}\n".format(key,results[key]))
w.close()

In [37]:
def create_embedding_method(model_name):
    def f(attribute,dataset,suffix,seed=random_seed):
        return load_shapley_vectors_simple(attribute,dataset,"_model_{}".format(model_name),seed=random_seed)
    return f

In [38]:
hierarchy_resnet = create_hierarchy(create_ward_hierarchy, load_shapley_vectors_simple,
                                    dataset,"_model_{}".format('resnet50'),attributes,43)

In [39]:
hierarchy_inception = create_hierarchy(create_ward_hierarchy, load_shapley_vectors_simple,
                                    dataset,"_model_{}".format('inceptionv3'),attributes,43)

In [40]:
print(hierarchy_resnet)

                ---- has_size::medium_(9_-_16_in)
           ----|
                ---- has_shape::duck-like

      ----|
                          ---- has_bill_shape::dagger
                     ----|
                          ---- has_crown_color::white

                ----|
                          ---- has_bill_length::about_the_same_as_head
                     ----|
                          ---- has_primary_color::black


           ----|
                          ---- has_nape_color::white
                     ----|
                               ---- has_upperparts_color::white
                          ----|
                               ---- has_back_color::white


                ----|
                               ---- has_bill_shape::hooked_seabird
                          ----|
                               ---- has_under_tail_color::white

                     ----|
                               ---- has_upper_tail_color::white
                          ----|
  

In [41]:
print(hierarchy_inception)

                          ---- has_upper_tail_color::buff
                     ----|
                          ---- has_under_tail_color::buff

                ----|
                               ---- has_upperparts_color::buff
                          ----|
                                    ---- has_wing_color::buff
                               ----|
                                    ---- has_back_color::buff


                     ----|
                               ---- has_nape_color::buff
                          ----|
                               ---- has_primary_color::buff



           ----|
                     ---- has_throat_color::buff
                ----|
                          ---- has_breast_color::buff
                     ----|
                               ---- has_underparts_color::buff
                          ----|
                               ---- has_belly_color::buff




      ----|
                     ---- has_belly_color::brown
          

### See how truthfulness hyperparameters impact things

In [43]:
dataset = CUB_Dataset()
attributes = dataset.get_attributes()

results = {}

for compare_concept in [1,3,5,7]:
    results[compare_concept] = truthfulness_metric_shapley(load_shapley_vectors_simple,dataset,attributes,
                                                           [43,44,45],model_name="VGG16",compare_concepts=compare_concept)

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 199s 5s/step
Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 199s 5s/step
Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 200s 5s/step
Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 200s 5s/step


In [44]:
w = open("results/evaluation/cub_truthfulness_ablation.txt","w")
for key in results:
    w.write("{}: {}\n".format(key,results[key]))
w.close()

## Evaluate the VAE Method

In [15]:
dataset = MNIST_Dataset()
attributes = dataset.get_attributes()
seeds = [43,44,45]

In [16]:
method = load_vae_vectors_simple
name = "VAE"
print("Computing {}".format(name))
start = time.time()

results = compute_all_metrics(method,
                                    dataset,
                                    attributes,
                                    seeds)
print("Method {}: {}".format(name,results))

name_lower = name.lower()
w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
for key in results:
    w.write("{}: {}\n".format(key,results[key]))
w.close()


Computing VAE
Stability: (0.04761904761904767, 0.0)
Image Robustness: (0.03174603174603178, 0.022447834323382487)
Image Responsiveness: (0.03174603174603178, 0.022447834323382487)
Found 10000 validated image filenames belonging to 10 classes.
Truthfulness: (0.9523809523809522, 1.1102230246251565e-16)
Method VAE: {'Stability': (0.04761904761904767, 0.0), 'Image Robustness': (0.03174603174603178, 0.022447834323382487), 'Image Responsiveness': (0.03174603174603178, 0.022447834323382487), 'Truthfulness': (0.9523809523809522, 1.1102230246251565e-16)}


In [17]:
dataset = CUB_Dataset()
attributes = dataset.get_attributes()
seeds = [43,44,45]

In [18]:
method = load_vae_vectors_simple
name = "VAE"
print("Computing {}".format(name))
start = time.time()

results = compute_all_metrics(method,
                                    dataset,
                                    attributes,
                                    seeds)
print("Method {}: {}".format(name,results))

name_lower = name.lower()
w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
for key in results:
    w.write("{}: {}\n".format(key,results[key]))
w.close()


Computing VAE
Stability: (0.9097222222222223, 0.01724022539383214)
Image Robustness: (0.2400793650793651, 0.30042657272349843)
Image Responsiveness: (0.4087301587301588, 0.2858830244671909)
Found 1198 validated image filenames belonging to 200 classes.
Truthfulness: (0.20952380952380953, 0.05051464168875505)
Method VAE: {'Stability': (0.9097222222222223, 0.01724022539383214), 'Image Robustness': (0.2400793650793651, 0.30042657272349843), 'Image Responsiveness': (0.4087301587301588, 0.2858830244671909), 'Truthfulness': (0.20952380952380953, 0.05051464168875505)}


## Evaluate Other CEM Vectors

In [8]:
dataset = CUB_Dataset()
attributes = dataset.get_attributes()
seeds = [43,44,45]

In [13]:
for method,name in zip([load_cem_loss_vectors_simple],['CEM_Loss']):
    print("Computing {}".format(name))
    start = time.time()

    results = compute_all_metrics(method,
                                        dataset,
                                        attributes,
                                        seeds)
    print("Method {}: {}".format(name,results))
    
    name_lower = name.lower()
    w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
    for key in results:
        w.write("{}: {}\n".format(key,results[key]))
    w.close()
    

Computing CEM_Loss
Stability: (0.9484126984126985, 0.02837370961522835)
Image Robustness: (0.9662698412698413, 0.029395897599263976)
Image Responsiveness: (0.9791666666666666, 0.008761667526118915)
Found 1198 validated image filenames belonging to 200 classes.


/auto/homes/njr61/main_code/src/metrics.py:359: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_concept_embeddings = np.array([embedding_method(i,dataset,"",seed=seed) for i in dataset.get_attributes()])


Truthfulness: (0.06964285714285713, 0.009560939526314888)
Method CEM_Loss: {'Stability': (0.9484126984126985, 0.02837370961522835), 'Image Robustness': (0.9662698412698413, 0.029395897599263976), 'Image Responsiveness': (0.9791666666666666, 0.008761667526118915), 'Truthfulness': (0.06964285714285713, 0.009560939526314888)}


In [14]:
for method,name in zip([load_cem_stratified_vectors_simple],['CEM_Stratified']):
    print("Computing {}".format(name))
    start = time.time()

    results = compute_all_metrics(method,
                                        dataset,
                                        attributes,
                                        seeds,metrics=[stability_metric],metric_names=['Stability'])
    print("Method {}: {}".format(name,results))
    
    name_lower = name.lower()
    w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
    for key in results:
        w.write("{}: {}\n".format(key,results[key]))
    w.close()
    

Computing CEM_Stratified
Stability: (0.7619047619047619, 0.1142898478088466)
Found 1198 validated image filenames belonging to 200 classes.


/auto/homes/njr61/main_code/src/metrics.py:359: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_concept_embeddings = np.array([embedding_method(i,dataset,"",seed=seed) for i in dataset.get_attributes()])


Truthfulness: (0.15654761904761905, 0.02568505834570407)
Method CEM_Stratified: {'Stability': (0.7619047619047619, 0.1142898478088466), 'Truthfulness': (0.15654761904761905, 0.02568505834570407)}
